```
補助本「ゼロから作るDeep Learning」
3章～7章までを輪読会の補足として、読んで行きます。
Eli Kaminuma 2018.1.31
```

### 補助本5章 [2018.2.26]
# 5章　誤差逆伝播法

- 前4章で数値微分で重みパラメータ勾配を求めた→時間かかる
- 5章で計算効率の高い「誤差逆伝播法」を、「数式」でなく「計算グラフ」で学ぶ。
- 誤差逆伝播法を計算グラフで学ぶアイデアは下記を参考
 - Andrej Karpathyのブログ [4] 
 - 彼と Fei-Fei Li 教授ら作：スタンフォード大学のディープラーニング授業「CS231n」[5]
 
 
 
- 目次

```
5.1 計算グラフ 
5.1.1 計算グラフで解く 
5.1.2 局所的な計算 
5.1.3 なぜ計算グラフで解くのか?

5.2 連鎖律
5.2.1 計算グラフの逆伝播 
5.2.2 連鎖律とは 
5.2.3 連鎖律と計算グラフ 

5.3 逆伝播 
5.3.1 加算ノードの逆伝播 
5.3.2 乗算ノードの逆伝播 
5.3.3 リンゴの例 

5.4 単純なレイヤの実装 
5.4.1 乗算レイヤの実装 
5.4.2 加算レイヤの実装 

5.5 活性化関数レイヤの実装 
5.5.1 ReLU レイヤ 
5.5.2 Sigmoid レイヤ 

5.6 Affine / Softmax レイヤの実装
5.6.1 Affine レイヤ
5.6.2 バッチ版 Affine レイヤ 
5.6.3 Softmax-with-Loss レイヤ·

5.7 誤差逆伝播法の実装 
5.7.1 ニューラルネットワークの学習の全体図 
5.7.2 誤差逆伝播法に対応したニューラルネットワークの実装 
5.7.3 誤差逆伝播法の勾配確認 
5.7.4 誤差逆伝播法を使った学習 
```

In [3]:
#前準備
import numpy as np
import sys, os, time
from collections import OrderedDict # 順序付きDict
import matplotlib.pyplot as plt

sys.path.append(os.pardir) # load_mnistするために親ディレクトリを追加。必要に応じて変更 or dataset.mnistを別途import
from dataset.mnist import load_mnist

In [4]:
# データの読み込み
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

Done
Done
Done
Done
Converting train-images-idx3-ubyte.gz to NumPy Array ...
Done
Converting train-labels-idx1-ubyte.gz to NumPy Array ...
Done
Converting t10k-images-idx3-ubyte.gz to NumPy Array ...
Done
Converting t10k-labels-idx1-ubyte.gz to NumPy Array ...
Done
Creating pickle file ...
Done!


In [5]:
## これまでの章で使った関数のインポート。 
## 本来は下記の2つのインポート文でも良いが、参照しやすいように記載
# from common.layers import *
# from common.gradient import numerical_gradient

# 3.2 章
def sigmoid(x):
    return 1 / (1 + np.exp(-x))    

# 4.4章より, 微分を使った勾配算出用 
def numerical_gradient(f, x):
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + h
        fxh1 = f(x) # f(x+h)
        
        x[idx] = tmp_val - h 
        fxh2 = f(x) # f(x-h)
        grad[idx] = (fxh1 - fxh2) / (2*h)
        
        x[idx] = tmp_val # 値を元に戻す
        it.iternext()   
        
    return grad

# 3章より
def softmax(x):
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis=0)
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T 

    x = x - np.max(x) # オーバーフロー対策
    return np.exp(x) / np.sum(np.exp(x))

# 4章より
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    # 教師データがone-hot-vectorの場合、正解ラベルのインデックスに変換
    if t.size == y.size:
        t = t.argmax(axis=1)
             
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

***

### 5.1 計算グラフ

- 計算グラフとは、計算の過程をグラフによって表したもの
- 順伝播（左→右）と逆伝播（右→左）
- 逆伝播は、微分を効率よく計算
- グラフとは、データ構造としてのグラフ(複数のノードとエッジで構成）を意味する


***

### 5.1.1 計算グラフで解く

```
問 1:太郎くんはスーパーで 1 個 100 円のリンゴを 2 個買いました。支払う金額
を求めなさい。ただし、消費税が 10% 適用されるものとします。
```

- 計算グラフはノードと矢印によって計算の過程を表す
- ノードは○で表記し、○の中に演算の内容を書く。
- 計算の途中結果を矢印の上部に書く(ノードごとの計算結果を左→右に表現)。
- FIG5-1   apple -(100)-> (x2) -(200)-> (x1.1) -(220)-> paid

次の問題は、加算ノード「＋」が加わり、りんごとミカンの金額を合算する。
```
問 2:太郎くんはスーパーでリンゴを 2 個、みかんを 3 個買いました。リンゴは
1 個 100 円、みかんは 1 個 150 円です。消費税が 10% かかるものとして、支払
う金額を求めなさい。
```

計算グラフの問題を解くには、

- 計算グラフを構築
- 計算グラフ上で計算を左から右へと進める(順伝播:forward propagation)
- 順伝播の逆を逆伝播(back propagation)と言う。

### 5.1.2    局所的な計算

- 計算グラフの特徴は、「局所的な計算」を伝播すること
- 局所的とは、「自分に関係する小さな範囲」
- 局所的な計算の結果を伝達することで、全体の複雑な計算の結果が得られる



### 5.1.3   なぜ計算グラフで解くのか?

- 計算グラフの利点
 - 最大の利点は、逆方向の伝播によって「微分」を効率良く計算できる点
 - 局所的な計算によって、各ノードでは単純な計算に集中（問題を単純化）
 - 計算グラフによって、途中の計算の結果をすべて保持

- 例：リンゴの値段が値上がりした場合、最終支払金額への影響は？
- これは「リンゴの値段に関する支払金額の微分」を求めることに相当
- リンゴの値段を x、支払金額を L とした場合、
  ∂L/∂x を求めることに相当

- 逆伝播は右から左へ「1 → 1.1 → 2.2」と微分の値が伝達
- 「リンゴの値段に関する支払金額の微分」の値は 2.2 
- リンゴが 1 円値上がりしたら、最終支払金額が 2.2 円増える
- 途中まで求めた微分(途中まで流れた微分)の結果を共有
- 計算グラフの利点は、順伝播と逆伝播で各変数の微分値を効率良く求める点



---

### 5.2 連鎖律

- 「局所的な微分」を伝達する原理は、連鎖律(chain rule)による。
- 連鎖律が計算グラフ上での逆伝播に対応することを明らかにする。

### 5.2.1 計算グラフの逆伝播

- 計算グラフを使った逆伝播の例
- y = f(x) という計算、この計算の逆伝播を図5-6に表す
- 逆伝播の計算手順は、信号Eに対して、ノードの局所的な微分
　(∂y/∂x)を乗算し、それを次のノードへ伝達していく。
- 局所的な微分とは、順伝播での y = f(x) という計算の微分を求めること
- xに関する y の微分(∂y∂x)を求めることを意味する。
- たとえば、y = f(x) = x^2 だとしたら、∂y/∂x = 2x 。
- 局所的な微分を上流から伝達された値(例では E)に乗算して、前ノードへと渡す。

### 5.2.2 連鎖律とは

- 合成関数 = 複数の関数によって構成される関数
- Z=(x+y)^2は、Z=t^2とt=(x+y)の合成関数

- 連鎖律とは合成関数の微分についての性質であり、次のように定義される

```
ある関数が合成関数で表される場合、合成関数の微分は、
合成関数を構成するそれぞれの関数の微分の積によって
表すことができる。
```

- これを連鎖律の原理と言う。
- 式 (5.1) の例で、∂z/∂xは、∂z/∂tと ∂t/∂xの積によって表す。

```
∂z/∂x = (∂z/∂t)*(∂t/∂x)

(∂z/∂t)= 2t
(∂t/∂x)=1

∂z/∂x = (∂z/∂t)*(∂t/∂x)= 2t*1=2(x+y)

```


### 5.2.3 連鎖律と計算グラフ

- 図5-7 式 (5.4) の計算グラフ:順方向とは逆向きの方向に、局所的微分を乗算して渡す
- 図5-7 で注目すべきは、一番左の逆伝播の結果。これは、連鎖律より、
(∂z/∂z)(∂z/∂t)(∂t/∂x) =(∂z/∂t)(∂t/∂x) =(∂z/∂x) が
成り立ち、「x に関する z の微分」に対応する。
つまり、逆伝播が行っていることは、連鎖律の原理から構成される。
- 図5-8 計算グラフの逆伝播の結果より、 ∂z/∂x は 2(x + y) 


### 5.3 逆伝播

- 前節では、計算グラフの逆伝播が連鎖律によって成り立つことを説明した
- 「+」や「×」などの演算を例に、逆伝播の仕組みについて説明する。
- 加算ノード： そのまま伝播. z = x + y の偏微分は共に1。
- 乗算ノード： ひっくり返した値(xならy,yならx)を乗算し伝播。

### 5.3.1 加算ノードの逆伝播

- 加算ノードの逆伝播は、z = x + y 対象に逆伝播を見る。

- 図 5-9 加算ノードの逆伝播:左図が順伝播、右図が逆伝播。右図の逆伝播が示すように、加算ノードの逆伝播は、上流の値をそのまま下流へ流す

- 図 5-10 最終的に出力する計算の一部に、今回の加算ノードが存在する。逆伝播の際には、一番右の出力からスタートして、局所的な微分がノードからノードへと逆方向に伝播されていく

- 加算の逆伝播具体例
-「10 + 5 = 15」という計算があるとして、逆伝播の際には、
 上流から 1.3 の値が流れてくるとします。これを計算グラフで書くと
 図5-11(加算ノードの逆伝播の具体例)
 - 加算ノードの逆伝播は入力信号を次のノードへ出力するだけなので、図5-11 のように、1.3 を次のノードへと流します。
 
 

### 5.3.2  乗算ノードの逆伝播

続いて、乗算ノードの逆伝播（z = xy を例に）

- 乗算の逆伝播の場合は、上流の値に順伝播の入力信号を“ひっくり返した値”を乗算して下流へ流す。
- ひっくり返した値(図5-12)
　- 順伝播の際に x の信号であれば逆伝播では y
  - 順伝播の際に y の信号であれば逆伝播では x 
- 具体例: 「10 × 5 = 50」
  - 逆伝播の際に上流から 1.3 の値が流れてくる
  - これを計算グラフで書くと図5-13 
  - 乗算の逆伝播は1.3 × 5 = 6.5、1.3 × 10 = 13 
  - なお加算の逆伝播では、上流の値をただ下流に流すだけ
  - 順伝播の入力信号の値は必要なし
  - 乗算の逆伝播は、順伝播のときの入力信号必要(信号保持)
  

### 5.3.3 リンゴの例

- リンゴ 2 個と消費税
- リンゴの値段、リンゴの個数、消費税の3変数が最終支払金額にどう影響するか。
- これは以下の3つを求める事に相当
  -「リンゴの値段に関する支払金額の微分」
  -「リンゴの個数に関する支払金額の微分」
  - 「消費税に関する支払金額の微分」
- これを計算グラフの逆伝播を使って解く（図5-14 ）
- 図 5-14 リンゴの買い物の逆伝播の例
  - 消費税とリンゴの値段が同じ量だけ増加したら、消費税は 200 の大きさで最終的な支払金額に影響を与え、リンゴの値段は 2.2 の大きさで影響を与える。
- 消費税とリンゴの値段はスケールが異なる
  - 消費税の1は100%
  - リンゴの値段の1は1円
- 図 5-15 リンゴとみかんの買い物の逆伝播の例
  - 四角に数字を入れて逆伝播を完成させよう

### 5.4 単純なレイヤの実装

実際にニューラルネットワークを構築するときに、各レイヤの組み合わせだけで簡単に計算できるようになる。

- 「リンゴの買い物」の例を、Python で実装
　　- 乗算ノードを「乗算レイヤ(MulLayer)」
    - 加算ノードを「加算レイヤ(AddLayer)」
-  次節ではニューラルネットワークの「層(レイヤ)」を1クラスで実装
  - 「レイヤ」とは、ニューラルネットワークにおける機能単位
  - 　下記はレイヤ単位で実装
     - シグモイド関数のための Sigmoid 
     - 行列の積のための Affine


## 5.4.1 乗算レイヤの実装

- レイヤは共通methodを持つ
   - forward() 順伝播 
   - backward() 逆伝播
- 乗算レイヤは MulLayer クラス
- ソー スコードはch05/layer_naive.py 

```
# coding: utf-8


class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None

    def forward(self, x, y):
        self.x = x
        self.y = y                
        out = x * y

        return out

    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x

        return dx, dy


class AddLayer:
    def __init__(self):
        pass

    def forward(self, x, y):
        out = x + y

        return out

    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1

        return dx, dy

```

- __init__() では、インスタンス変数 (x,y) を初期化
- 順伝播時の入力値を保持。
  - forward()では(x,y)の2引数を乗算
  - backward()では
      - 上流からの微分(dout)に対し順伝播の“ひっくり返した値”を乗算
      - 下流に流す 
 - リンゴ 2 個と消費税の計算グラフ（図5-16参照）

In [7]:
# レイヤーを定義(順伝播、逆伝播)

#--乗算レイヤ---------------------------
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None

    def forward(self, x, y):  #順伝播
        self.x = x
        self.y = y                
        out = x * y

        return out

    def backward(self, dout):  #逆伝播
        dx = dout * self.y
        dy = dout * self.x

        return dx, dy


#---加算レイヤ------------------------------
class AddLayer:
    def __init__(self):
        pass

    def forward(self, x, y):
        out = x + y

        return out

    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1

        return dx, dy

In [13]:
#-----------------------------------------------
# 乗算レイヤ(MulLayer)の例(2つのリンゴを購入,消費税1.1)
#-------------------------------------------------
from layer_naive import *

apple = 100
apple_num = 2
tax = 1.1

mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

# backward
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print("total price:", int(price))
print("apple:", apple)
print("apple_num:", apple_num)
print("dApple:", dapple)
print("dApple_num:", int(dapple_num))
print("dTax:", dtax)
print("----------------")
print("dapple * apple = " + ('%0.0f' % (dapple * apple)) )
print("dapple_num * apple_num = " + ('%0.0f' % (dapple_num * apple_num)))
print("dTax * tax = " + ('%0.0f' % (dtax * tax)) )

total price: 220
apple: 100
apple_num: 2
dApple: 2.2
dApple_num: 110
dTax: 200
----------------
dapple * apple = 220
dapple_num * apple_num = 220
dTax * tax = 220


In [18]:
# backward　
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)
print('dapple=',dapple)
print('dapple_num=' + ('%0.0f'% dapple_num))
print('dtax=',dtax) # 2.2

# backward() の呼び出す順番は、forward() と逆の順番
# backward() の引数は、「順伝播の際の出力変数に対する微分」を入力
# たとえば、mul_apple_layer という乗算レイヤは、
#   - 順伝播時にapple_price を出力
#   - 逆伝播時にapple_price の微分値dapple_price を引数に設定

dapple= 2.2
dapple_num=110
dtax= 200


---

## 5.4.2 加算レイヤ(AddLayer)の実装

```
from layer_naive import *

apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# layer
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)  # (1)
orange_price = mul_orange_layer.forward(orange, orange_num)  # (2)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)  # (3)
price = mul_tax_layer.forward(all_price, tax)  # (4)

# backward
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)  # (4)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)  # (3)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)  # (2)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)  # (1)

print("price:", int(price))
print("dApple:", dapple)
print("dApple_num:", int(dapple_num))
print("dOrange:", dorange)
print("dOrange_num:", int(dorange_num))
print("dTax:", dtax)


#　加算レイヤでは初期化は必要ないので、__init__() では何も行わず
#　(pass 記述は「何も行わない」という命令)
# 加算レイヤの forward()では2 引数(x,y)を受け取り、加算して出力
# backward() では上流から伝わってきた微分(dout)を、そのまま下流に流すだけ

```

## 加算レイヤを乗算レイヤと組み合わせ

In [20]:
#- リンゴ 2 個とみかん 3 個の買い物を実装(図5-17)
#  - 消費税1.1
#  - 加算レイヤと乗算レイヤ
#  - ソースコードは ch05/buy_apple_orange.py
from layer_naive import *

apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# layer
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()# 加算レイヤ加わった図5-15
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)  # (1)
orange_price = mul_orange_layer.forward(orange, orange_num)  # (2)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)  # (3)
price = mul_tax_layer.forward(all_price, tax)  # (4)

# backward
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)  # (4)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)  # (3)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)  # (2)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)  # (1)

print("price:", int(price))
print("dApple:", dapple)
print("dApple_num:", int(dapple_num))
print("dOrange:", dorange)
print("dOrange_num:", int(dorange_num))
print("dTax:", dtax)


price: 715
dApple: 2.2
dApple_num: 110
dOrange: 3.3000000000000003
dOrange_num: 165
dTax: 650


---

---

## 5.5 活性化関数レイヤの実装

- 計算グラフの考え方をニューラルネットワークに適用
- ニューラルネットワークを構成する「層(レイヤ)」を1クラスとして実装する
- まずは、活性化関数である ReLU と Sigmoid レイヤ（143-146参照）を実装


## 5.5.1 ReLU レイヤ

- 活性化関数 ReLU(Rectified Linear Unit)


$
\begin{equation}
\qquad \quad      y=x (x>0), 0 (x \leq 0)
\end{equation}
$

上式の微分

$
\begin{equation}
\qquad \quad      \partial y/\partial x=1 (x>0), 0 (x \leq 0)
\end{equation}
$

- 順伝播時の入力である x が 0 より大きければ、逆伝播は
 上流の値をそのまま下流に流します。
- 逆に、順伝播時に x が 0 以下であれば、逆伝播では
  下流への信号はそこでストップします。
- 計算グラフで表すと図5-18

-  ReLU レイヤの実装
 - ニューラルネットワークのレイヤの実装では、forward() や
   backward() の引数には、NumPy の配列が入力されることを想定
 - ReLU レイヤの実装プログラムはcommon/layers.py 


In [11]:
class Relu:
    def __init__(self):
        self.mask = None
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()   #ランプ関数の実装
        out[self.mask] = 0
        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout # 1
        return dx


Relu クラスは、インスタンス変数として mask という変数を
持ちます。この mask変数は True/False からなるNumPy 配列で、
順伝播の入力である x の要素で 0 以下の場所を True、
それ以外(0 より大きい要素)を False として保持します。たとえば、
次の例で示すように、True/False からなる NumPy 配列を mask
変数は保持します。

In [13]:
import numpy as np
x = np.array( [[1.0, -0.5], [-2.0, 3.0]] )
print(x)
mask = (x <= 0)
print(mask)


[[ 1.  -0.5]
 [-2.   3. ]]
[[False  True]
 [ True False]]


図5-18 で示すように、順伝播時の入力の値が 0 以下ならば、
逆伝播の値は 0 になります。そのため、逆伝播では、順伝播時に
保持した mask を使って、上流から伝播された dout に対して、
mask の要素が True の場所を 0 に設定します。

ReLU レイヤは、回路における「スイッチ」のように機能します。順伝播時に電
流が流れていればスイッチを ON にし、電流が流れなければスイッチを OFF
にします。逆伝播時には、スイッチが ON であれば電流がそのまま流れ、OFF
であればそれ以上電流は流れません。

## 5.5.2 Sigmoid レイヤ


#- シグモイド関数を実装 (式5.9) 

y =1/ (1 + exp(−x))

- 式 (5.9) を計算グラフで表す(図5-19)
- 逆伝播の流れ

- ステップ 1
「/」ノードは y =1x を表す。この微分は解析的に次式。∂y/∂x = −1/x^2= −y^2

- ステップ 2
「+」ノードは、上流の値を下流にそのまま流すだけ

- ステップ 3
「exp」ノードは y = exp(x) を表し、その微分は次の式で表されます。
計算グラフでは、上流の値に対して、順伝播時の出力――
この例では exp(−x)――を乗算して下流へ伝播。

- ステップ 4
「×」ノードは、順伝播時の値を“ひっくり返して”乗算。そのため、ここ
では −1 を乗算

- 逆伝播の出力は ∂L/∂y * y^2 * exp(−x) （図5-20 結果）
- この値が下流にあるノードに伝播
- ∂L/∂y y^2 exp(−x) という値が順伝播の入力 x
- 図5-20 の計算グラフは、図5-21がグループ化した「sigmoid」ノード

- 図5-20 の計算グラフと図5-21 の簡略版の計算グラフは、計算の結果は同じ
- 簡略版の計算グラフのほうが、逆伝播の際の途中の計算を省略することができる
- また、ノードをグループ化することによって、Sigmoid レイヤの中身は気にならず

- ∂L/∂y y^2 exp(−x) は∂L/∂y y(1 − y)

- 図5-21 Sigmoid レイヤの逆伝播は、順伝播の出力だけから計算可能

In [ ]:
- Sigmoid レイヤを Python で実装（図5-22）
- common/layers.py

In [15]:
class Sigmoid:
    def __init__(self):
        self.out = None
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        return out
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out #偏微分した式(y*(1-y))
        return dx

- 順伝播時=出力をインスタンス変数の out に保持
- 逆伝播時=out 変数を使って計算を行う

---

---

## 5.6 Affine / Softmax レイヤの実装
### 5.6.1 Affine レイヤ

- Y = np.dot(X, W) + B
- 行列の積の計算は、要素数を一致させる必要有
- X(2,)・W(2,3) = O (3,)　、括弧内は要素数
- ニューラルネットワークの順伝播で行う行列の積は、幾何学の分野では「ア
フィン変換」と呼ばれる。ここでは、アフィン変換を行う処理を
「Affine レイヤ」という名前で実装する。

- np.dot(X, W) + B の計算グラフ(図5-24)
- これまで見てきた計算グラフは「スカラ値」がノード間を流れた
- この例では「行列」がノード間を伝播


- 図5-24計算グラフの逆伝播について考える。
- (∂L/∂X) =(∂L/∂Y)· W^T
- (∂L/∂W) = X^T ·(∂L/∂Y)

- T:転置
- 図 5-25 Affineレイヤの逆伝播:変数が多次元配列。逆伝播の各変数下部に。

- Xと (∂L/∂X) 、Wと(∂L/∂W) は同じ形状(=要素数)に注意

- X = (x0, x1, ··· , xn)
- (∂L/∂X) =( ∂L/∂x0, ∂L/∂x1, ··· , ∂L/∂xn)

- なぜ行列の形状に注意するか
　　- 行列の積では、対応する次元の要素数を一致させる必要あり

- 図 5-26 = 行列の積(dotノード)の逆伝播
    - 行列の対応次元の要素数を一致させるように積を組み立てることで導く

### 5.6.2 バッチ版 Affine レイヤ

- N 個のデータをまとめて順伝播する場合を考える
- つまり、バッチ版の Affine レイヤ
- (データのまとまりは「バッチ」と呼ぶ)

- 図5-27=バッチ版のAffine レイヤ

```
先ほどの説明と異なる点は、入力である X の形状が (N, 2) になっただけ.
逆伝播の際は、行列の形状に注意すれば、∂L/∂X と ∂L/∂W は前と
同じように導出する。
```

- バイアスの加算に際しては、注意が必要
 - 順伝播の際のバイアスの加算は、X · W に対して、バイアスがそれぞれのデータに加算されます。
 - N=2(データが 2 個)とした場合、バイアスは、その 2 個のデータ
  それぞれに対して(それぞれの計算結果に対して)加算される。
  

In [22]:
#-------------------------------
# バイアスの加算 (順伝播)
#--------------------------------
import numpy as np

X_dot_W = np.array([[0, 0, 0], [10, 10, 10]])
B = np.array([1, 2, 3])

tmp = X_dot_W

Y = X_dot_W+B

print('X_dot_W=', tmp)
print('X_dot_W+B=', Y)

X_dot_W= [[ 0  0  0]
 [10 10 10]]
X_dot_W+B= [[ 1  2  3]
 [11 12 13]]


In [3]:
#-------------------------------
# バイアスの加算 (逆伝播)
#--------------------------------
import numpy as np

dY =  np.array([[1, 2, 3,], [4, 5, 6]])

dB = np.sum(dY,axis=0)

print('dY=', dY)
print('dB=', dB)

#この例では、データが 2 個(N=2)と仮定
# バイアスの逆伝播は、2 個データに対しての微分を、データごとに合算
# np.sum() で、0 番目の軸(データを単位とした軸)に対して (axis=0)の総和計算

dY= [[1 2 3]
 [4 5 6]]
dB= [5 7 9]


In [4]:
#---------Affine の実装-----------------
#
# common/layers.py 
# 入力データがテンソル(4 次元のデータ)の場合も考慮した実装
#----------------------------------

class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
    def forward(self, x):
        # tensor対応
        self.original_x_shape=x.shape
        x=x.shape ##### NOTE!!!
        self.x = x
        out = np.dot(x, self.W) + self.b
        return out
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        return dx


### 5.6.3 Softmax-with-Loss レイヤ

- 最後に、出力層ソフトマックス関数
  - Softmax レイヤは、入力された値を正規化(出力の和が
  1になるように変形)して出力。
  - 手書き数字認識（図5-28）
  - 手書き数字認識は、10 クラス分類を行うため、Softmax 
  レイヤへの入力は 10 個ある。
- ニューラルネットワークは、推論(inference)と学習の2フェーズ
  - ニューラルネットワークの推論では、通常、Softmax レイヤは
   使用しない。図5-28 のネットワークで推論を行う場合、
   最後の Affine レイヤの出力を認識結果として用いる。
  - ニューラルネットワークの正規化しない出力結果
   (図5-28 Softmax前層の Affine レイヤ出力)は「スコア」と呼ぶ   
  - ニューラルネットワークの推論で答えを1つだけ出す場合は、
　スコア最大値だけに興味があるため、Softmaxレイヤは必要ない
  - 一方、ニューラルネットワークの学習時には、Softmax レイヤが
  必要になる
  
- 図 5-28 入力画像がAffine レイヤと ReLU レイヤによって変換され、
  Softmax レイヤによって 10個の入力が正規化される。
  - 「0」スコアは5.3、Softmax レイヤによって 0.008(0.8%)に変換される
  - 「2」であるスコアは 10.1 であり 0.991(99.1%)に変換される  
  

- 「Softmax-with-Loss レイヤ」実装
  - 損失関数である交差エントロピー誤差(cross entropy error)も含める
  - 図5-29=Softmax-with-Loss レイヤ(ソフトマックス関数と交差エントロピー誤差)の計算グラフ
  - 付録A = Softmax-with-Loss レイヤの導出過程

- 図 5-30= 「簡易版」Softmax-with-Loss レイヤの計算グラフ
  - Softmax レイヤは、入力である (a1, a2, a3) を正規化
  - (y1, y2, y3) を出力
  - Cross Entropy Error レイヤは、Softmax の出力 (y1, y2, y3) と、
   教師ラベルの (t1, t2, t3) を受け取り、損失 Lを出力
   
- 注目すべきは、図5-30逆伝播の結果
  - Softmax レイヤからの逆伝播は(y1 − t1, y2 − t2, y3 − t3)“キレイ”な結果
  - (y1, y2, y3) はSoftmax レイヤの出力、(t1, t2, t3) は教師データ
  - (y1 − t1, y2 − t2, y3 − t3)は、Softmax レイヤの出力と教師ラベルの差分になる。
  - ニューラルネットワークの逆伝播では、この差分誤差が前レイヤへ伝わる(学習の重要性質)
 

- ニューラルネットワークの学習の目的は、ニューラルネットワーク
  の出力(Softmax の出力)を教師ラベルに近づけるように、重みパラメータを調整すること。
- 先ほどの (y1 − t1, y2 − t2, y3 − t3) 結果は、まさに Softmax レイヤ
  の出力と教師ラベルの差であり、現在のニューラルネットワークの出力と
  教師ラベルの誤差を素直に表している
 
  - 実は“キレイ”な結果になる様に、交差エントロピー誤差が設計された。
  - 回帰問題では出力層に「恒等関数」を用い、損失関数として「2 乗和誤差」を用いる(「3.5 出力層の設計」参照)、
　　これも同様の理由によります。つまり、「恒等関数」の損失関数として「2 乗和誤差」を用いると、逆伝播が
  　(y1 − t1, y2 − t2, y3 − t3) という“キレイ”な結果になる

- 具体例
　　- 教師ラベル (0, 1, 0) 、Softmax レイヤの出力 (0.3, 0.2, 0.5) の場合
      - 正解ラベルに対する確率は 0.2(20%)
      - この時点のニューラルネットワークは正しい認識ができていない
      - この場合、Softmax レイヤからの逆伝播は、(0.3, −0.8, 0.5) という大きな誤差を伝播する
　　- 教師ラベル (0, 1, 0) 、Softmax レイヤの出力 (0.01, 0.99, 0) の場合
  　　- このニューラルネットワークは、かなり正確に認識
      - Softmax レイヤからの逆伝播は、(0.01, −0.01, 0) という小さな誤差に
      - この小さな誤差が前レイヤに伝播していくが、誤差は小さいためSoftmax レイヤより前にあるレイヤが学習する内容も小さくなる

In [23]:
#-------------------------------------------------------------------------------------
# Softmax-with-Loss 
#   - Softmaxと交差エントロピー誤差をまとめて実装
#   - 逆伝播の値がy-tという非常にキレイな値となる
#   - Softmaxには交差エントロピー誤差を、恒等関数の場合は二乗和誤差を損失関数として使う
#--------------------------------------------------------------------------------------
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None # 損失
        self.y = None # softmax の出力
        self.t = None # 教師データ(one-hot vector)
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        return dx

# 逆伝播の際には、伝播する値をバッチの個数(batch_size)で割ることで、
# データ 1 個あたりの誤差が前レイヤへ伝播する点に注意

---

---

# 5.7 誤差逆伝播法の実装

- 前節で実装したレイヤを組み合わせ、レゴブロックを作るようにニューラルネットワークを構築する
    
## 5.7.1 ニューラルネットワークの学習の全体図

- 前提
ニューラルネットワークは、適応可能な重みとバイアスがあり、この重みとバ
イアスを訓練データに適応するように調整することを「学習」と呼ぶ。ニュー
ラルネットワークの学習は次の 4手順で行う。

  - ステップ 1(ミニバッチ)
    訓練データ中からランダムに一部データを選出

  - ステップ 2(勾配の算出)
    各重みパラメータに関する損失関数の勾配を計算

  - ステップ 3(パラメータの更新) 
   重みパラメータを勾配方向に微小量だけ更新

  - ステップ 4(繰り返す)

  - ステップ 1、ステップ 2、ステップ 3 を繰り返す


## 5.7.2 誤差逆伝播法に対応したニューラルネットワークの実装

- 2 層のニューラルネットワークをTwoLayerNet として実装
- クラスのインスタンス変数とメソッドを整理 (表5-1 と表5-2 )

- レイヤを使用することによって、認識結果を得る処理(predict())
や勾配を求める処理(gradient())がレイヤの伝播だけで達成

In [37]:
# coding: utf-8
import sys, os
sys.path.append('./')  # 親ディレクトリのファイルをインポートするための設定
import numpy as np
from saitobook.common.layers import *
from saitobook.common.gradient import numerical_gradient
from collections import OrderedDict


class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        # 重みの初期化
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size) 
        self.params['b2'] = np.zeros(output_size)

        # レイヤの生成（単純計算のまとまり）
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        #--tが必要なので上と分けた
        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        
        return x
        
    # x:入力データ, t:教師データ
    #------------------------------------------------
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    # x:入力データ, t:教師データ
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
        
    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward----------------------------------
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        #--------------------------------------------    
        # 設定
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads


- 特にニューラルネットワークのレイヤを OrderedDict として保持する点が重要

- OrderedDict=順番付きのディクショナリ
  - 「順番付き」とは、ディクショナリに追加した要素の順番を覚える
  - ニューラルネットワークの順伝播では、追加した順にレイヤの forward() メソッドを呼び出すだけで処理が完了
- 逆伝播では、逆の順番でレイヤを呼び出すだけ
  - Affine レイヤやReLU レイヤが、内部で順伝播と逆伝播を正しく
   処理してくれる
  - ここで行うことはレイヤを正しい順番で連結し、順番に(or逆順に)レイヤを呼び出すだけ



## 5.7.3 誤差逆伝播法の勾配確認


- これまで勾配を求める方法を2つ説明してきた。
  - 1) 数値微分によって求める方法
  - 2) 解析的に数式を解いて求める方法

- これからは誤差逆伝播法で勾配を求めることにしましょう。
  - 後者の解析的に求める方法について言えば、誤差逆伝播法を用いることで、大量のパラメータが存在しても効率的に計算できた
  - 計算に時間のかかる数値微分ではなく、誤差逆伝播法で勾配を求めることにしましょう

- 数値微分の利点は、実装が簡単であるということ
  - 勾配確認(gradient check)= 数値微分で勾配を求めた結果と、誤差逆伝播法で求めた勾配の結果が一致すること
    を確認する作業
  -(ソースコードは ch05/gradient_check.py)



In [43]:
#-------------------------------------------------
# 実行時間差を計算する(多田さんscriptより)
#-------------------------------------------------
import time
class Timer:
    def __init__(self):
        self.start = time.time()
        self.lap_start = self.start
        self.lap_list = []

    def lap_show (self):
        now = time.time()
        lap = now - self.lap_start
        self.lap_list.append(lap)
        self.lap_start = now
        print("lap " + str(len(self.lap_list)) + ((": %0.7f sec") %lap) )

In [61]:
#-------------------------------------
#  誤差逆伝播法(original)
#-------------------------------------
import sys, os
sys.path.append('./')  # 親ディレクトリのファイルをインポートするための設定
import numpy as np
from saitobook.dataset.mnist import load_mnist
from saitobook.samples.two_layer_net_mod import TwoLayerNet

# データの読み込み
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

#---------------------------------------------
timer = Timer()
print("実行速度 lap1: 数値微分、lap2: 誤差逆伝播法")
#--------------------------------------------
grad_numerical = network.numerical_gradient(x_batch, t_batch)
timer.lap_show()

#--------------------------------------------
grad_backprop = network.gradient(x_batch, t_batch)
timer.lap_show()
#--------------------------------------------
for key in grad_numerical.keys():
    diff = np.average( np.abs(grad_backprop[key] - grad_numerical[key]) )
    print(key + ":" + str(diff))
    
#--------------------------------------------    
print('数値微分time=%e' % timer.lap_list[0])
print('誤差逆伝播time=%e' % timer.lap_list[1])
if timer.lap_list[1] > 0:
    print(("数値微分/誤差逆伝播法=%0.1f倍" % (timer.lap_list[0] / timer.lap_list[1])))


実行速度 lap1: 数値微分、lap2: 誤差逆伝播法
lap 1: 6.8508735 sec
lap 2: 0.0000000 sec
W1:2.0390068474371752e-10
b1:8.501201741802251e-10
W2:6.955964897092937e-08
b2:1.3883158105770788e-07
数値微分time=6.850873e+00
誤差逆伝播time=0.000000e+00
